## Find Missing Bounding Box for UMedia Maps Metadata

If you have already got the metadata CSV file after running the `harvest.py` and found the **Bounding Box** field are empty for all records, you may need this script to help you find those missing Bounding Boxes. However, if you can find the Bounding Box values, then just igonore this script.

> Original created on Dec 1, 2020 by Gene Cheng

> Updated on May 1st, 2021 by Gene Cheng

> Updated on Nov 8th, 2021 by Gene Cheng

### STEP 1. Go and Download the Lookup Sheet

In order to find the bounding box, we need to download a sheet which may includes both **Identifier** and **Coordinate** fields for the maps you harvested just now. Here's the [link](https://docs.google.com/spreadsheets/d/1A2MxmWxQ31_aDpxYQ5WSS9WcuoGdLGa9MOJWF-M7q6U/edit#gid=837746637) to a spreadsheet from Map Library and you can find there are more than one sheets.

Usually, I will copy some random `Identifiers` from the metadata CSV and use the **Find and Replace** tool from Google Sheet to search for these Identifiers among all exsiting sheets, and the one which contains these Identifiers is the sheet you need to download next. FYI, I downloaded the one called **Master List** for this script. But the data are updated frequently, so you may need to download right before you execute this script.

After you download, please make sure to move it into the current directory and rename it to `bbox_lookup.csv` instead.


### STEP 2. Read the Lookup Sheet

Since we only need the **System Identifier** and the **Coordinates** fields from the lookup sheet, we need to confirm the index of these two columns before reading the file. FYI, the index of **System Identifier** is **41**, and the index of **Coordinates** is **28**.

Then, read and extract these two fields only. Some values may have leading or trailing spaces, so that we need to use the `strip()` to remove them before store into a dictionary.

In [1]:
import csv
import re

In [24]:
bbox_lookup = {}
with open('bbox_lookup.csv') as fr:
    reader = csv.reader(fr)
    fields = next(reader)
    for row in reader:
        ID = row[41].strip()
        coords = row[28].strip()
        bbox_lookup[ID] = coords

### STEP 3. Clean Data

This step might be the most important process through the whole script, since the format from the lookup sheet are 4 **Degree, Minute, Second** coordinates connected by some characters like **/** and **--**. For example: `(E 12°00'--E 48°00'/N 57°00'--N 47°00').` 

So first of all, we will use the `clean_coords()` function to remove the parentheses and the period, and separate it into 4 coordinates individually. 



In [8]:
# Remove parentheses and the period characters
# Separate four coordiantes
def clean_coords(coords):
    coords = re.sub(r'[().]', '', coords)
    [lon_range, lat_range] = coords.split('/')
    try:
        try:
            [w, e] = lon_range.split('--')
            [n, s] = lat_range.split('--')
        except:
            [w, e] = lon_range.split('-')
            [n, s] = lat_range.split('-')
    except:
        return [lon_range, lat_range]
        
    return [w, e, n, s]

Then, we will continue using the `parse_dms()` function to parse each coordinate into **Direction, Degree, Minute and Second** separately.


In [9]:
# Parse a DMS coordinate into different parts: direction, Degree, Minute, Second
def parse_dms(dms):
    dms = dms.strip()
    parts = re.split('[^\d\w]+', dms)[:-1]
    dir = ''
    deg = min = sec = 0
    if len(parts) == 4:
        [dir, deg, min, sec] = parts
        
    if len(parts) == 3:
        [dir, deg, min] = parts
    
    if len(parts) == 2:
        [dir, deg] = parts

    return (dir, deg, min, sec)

Last, convert the coordinate from **Degree,Minute,Second** to **Decimal Degree** with 4 digits after decimal point.

In [10]:
# Convert a coordiante from DMS format to Decimal Degree
def dms2dd(parsed_dms):
    (direction, degree, minute, second) = parsed_dms
    dd = float(degree) + float(minute)/60 + float(second)/(60*60)
    if direction == 'S' or direction == 'W':
        dd *= -1
    return round(dd,4)
    

In [31]:
bbox_dd = {}
for ID in bbox_lookup:
    coords = bbox_lookup[ID]
    if not coords:
        continue
    dds = []  
    for coord in clean_coords(coords):
        dd = dms2dd(parse_dms(coord))
        dds.append(dd)
    try:
        [w, e, n, s] = dds
        bbox = ','.join([w,s,e,n])
    except:
        bbox = ','.join([ str(x) for x in dds])
    
    bbox_dd[ID] = bbox

### STEP 4. Find Missing BBOX for UMedia Maps

Before running the following cells, please **edit** the first cell of Step 4 to the change the filename of the metadata CSV file from reaccession.

If more than one csv files exist, change the filename and rerun the step 4&5 again.

In [59]:
# Hello, please edit here !!
filename = 'reports/dateAdded_202110.csv'

In [60]:
records = []
with open(filename) as fr:
    reader = csv.reader(fr)
    fields = next(reader)
    for row in reader:
        records.append(row)

In [61]:
for record in records:
    ID = record[16].strip()
    if ID in bbox_dd:
        record[11] = bbox_dd[ID]

### STEP 5. Write a new CSV

Write to a new CSV file with the same name to overwrite the previous one.

In [62]:
with open(filename, 'w') as fw:
    writer = csv.writer(fw)
    writer.writerow(fields)
    writer.writerows(records)